In [1]:
epsilon = '\u03B5'
arrow = ' \u2192 '
import pandas as pd

In [2]:
def printGrammar(Grammar):
    lhs = list(set([t[0] for t in Grammar]))
    for l in lhs:
        rhs = [t[1] for t in Grammar if t[0] == l]
        if '' in rhs :
            rhs.remove('')
            rhs.append(epsilon)
        b = ' | '.join(rhs)
        print('{0:<3} {2} {1:<25}'.format(l,b,arrow))

In [3]:
def First(X,G):
    first = []
    nt = list(set([t[0] for t in G]))
    if X == '':
        first.append(epsilon)
        return first
    
    for x in X: 
        prod = [t[1] for t in G if t[0]==x]
        
        if x not in nt:
            first.append(x)
            if epsilon in first:
                first.remove(epsilon)
            return first

        if '' in prod:
            first.append(epsilon)
            
        for p in prod:
            for i in p:
                if i not in nt:
                    first.append(i)
                    break
                else:
                    pr = [t[1] for t in G if t[0]==i]
                    first += First(i,G)
                    if '' in pr:
                        if i != p[-1]:
                            first.remove(epsilon)
                        continue
                    break
        if epsilon not in first:
            break
            
    return list(set(first))


In [4]:
def Follow(x,G,start):
    nt = list(set([t[0] for t in G]))
    follow = []
    if x not in nt:
        return -1
    if x == start:
        follow.append('$')
    for a,b in G:
        if x in b:
            spl = b.split(x,maxsplit=1)
            follow += First(spl[1],G)
            if epsilon in follow:
                follow.remove(epsilon)
            if epsilon in First(spl[1],G):
                follow += Follow(a,G,start)
    return list(set(follow))


In [5]:
def AugmentedGrammar(G,start):
    G.append((start+"'",start))
    return G

In [6]:
def Closure(item,G):
    for i in item:
        a,b = i[1].split('.')
        if b!='':
            if len(b)==1:
                la = i[2]
            else:
                la = tuple(First(b[1],G))
            for j in G:
                if j[0] == b[0]:
                    item.append((j[0],'.'+j[1],la))
    return item

In [7]:
def goto(item,x,G):
    new = []
    for i in item:
        a,b = i[1].split('.')
        if b[0]==x:
            new.append((i[0],a+x+'.'+b[1:],i[2]))
    return Closure(new,G)     

In [8]:
def PossibleMoves(item):
    mov = []
    for i in item:
        a,b = i[1].split('.')
        if not b == '':
            mov.append(b[0])
    return mov

In [9]:
def LR0set(G,start):
    aG = AugmentedGrammar(G,start)
    lr0 = [(start+"'",'.'+start,tuple('$'))]
    return Closure(lr0,G)    

In [10]:
def CannonicalCollection(G,start):
    old = []
    lr0 = set(LR0set(G,start))
    old.append(lr0)
    change = True
    completed = []
    while change:
        new = old.copy()
        for i in new:
            if i not in completed:
                mov = PossibleMoves(i)
                for x in mov:
                    temp= set(goto(i,x,G))
                    if temp not in completed:
                        new.append(temp)
                completed.append(i)

        if old == new:
            change = False
        else:
            old = new
    
    return completed

In [11]:
def ParsingTable(G,start):
    nt = list(set([t[0] for t in G]))
    ter = list(set([t[1] for t in G if t[1] not in nt ]))
    cc = CannonicalCollection(G,start)
    cc = {a:b for a,b in enumerate(cc)}
    t = []
    for i in ter:
        if i != '':
            for j in i:
                if j not in nt:
                    t.append(j)
    t.append('$')
    t = list(set(t))
    it = list(cc.keys())
    action = pd.DataFrame(index=it, columns=t)
    action = action.fillna('')
    Goto = pd.DataFrame(index=it, columns=nt)
    Goto = Goto.fillna('')
    
    for i in it:
        item = cc[i]
        for j in item:
            a,b = j[1].split('.')
            if b == '':
                if a == start and j[0] == start+"'":
                    action['$'][i] = 'ACCEPT'
                    
                else:
                    for k in action.columns:
                        if k in j[2]:
                            action[k][i] = action[k][i] + ' REDUCE({0} {2} {1}) '.format(j[0],a,arrow) 
                        
        mov = PossibleMoves(item)
        for j in mov:
            k = [key for (key,value) in cc.items() if value == set(goto(item,j,G))][0]
            if j in nt:
                Goto[j][i] = k
            elif j!='$':
                action[j][i] = action[j][i] + ' SHIFT({}) '.format(k) 
                
                                
    LALR = dict()
    LALR['action'] = action
    LALR['goto'] = Goto
    
    return LALR,cc

In [12]:
G = [('S','AA'),('A','aA'),('A','b')]
start = 'S'
printGrammar(G)
LALR,cc = ParsingTable(G,start)

A    →  aA | b                   
S    →  AA                       


In [13]:
LALR['action']

,b,a,$
0,SHIFT(1),SHIFT(2),
1,REDUCE(A → b),REDUCE(A → b),
2,SHIFT(1),SHIFT(2),
3,,,ACCEPT
4,SHIFT(7),SHIFT(6),
5,REDUCE(A → aA),REDUCE(A → aA),
6,SHIFT(7),SHIFT(6),
7,,,REDUCE(A → b)
8,,,REDUCE(S → AA)
9,,,REDUCE(A → aA)


In [14]:
LALR['goto']

,A,S
0,4,3
1,,
2,5,
3,,
4,8,
5,,
6,9,
7,,
8,,
9,,


In [25]:
# Parsing Procedure

string = "aabb"
string = string + '$'

stack = [0]
ptr = 0
while True:
    s = stack[-1]
    q = string[ptr]
    
    action = LALR['action'].loc[s][q]
    
    if action == 'ACCEPT':
        print('\n ACCEPT')
        break
    elif action == '':
        print('Parsing Error !')
        break
    elif 'SHIFT' in action:
        ns = int(action.split('(')[1].split(')')[0])
        stack.append(q)
        stack.append(ns)
        ptr = ptr + 1
    else:
        print(action)
        res = action.split('(')[1].split(')')[0].split(' ')
        pop_len = 2*len(res[-1])
        for i in range(pop_len):
            stack.pop()
        stack.append(res[0])
        goto = LALR['goto'].loc[stack[-2]][stack[-1]]
        stack.append(goto)

 REDUCE(A  →  b) 
 REDUCE(A  →  aA) 
 REDUCE(A  →  aA) 
 REDUCE(A  →  b) 
 REDUCE(S  →  AA) 

 ACCEPT
